Домашнее задание №1
 Гадоев Рахмонали 
 1.На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.
 2.На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.
 3.На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.
 4.На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.
 5.На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.


In [59]:
from connect import set_connection
from sqlalchemy import text
import pandas as pd
import matplotlib as plt
import plotly.express as px
import statsmodels.api as sm

In [2]:
with set_connection() as pg:
    pg.execute(text("select 1"))

In [5]:
query = text("select * from track limit 100")
with set_connection() as conn:
    df = pd.read_sql(query, conn)

1.На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.

In [30]:
query = text("""
    SELECT 
        g.name AS genre,
        ROUND(SUM(t.milliseconds) / 60000) AS total_minutes
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
    ORDER BY total_minutes DESC
""")
with set_connection() as conn:
    df = pd.read_sql(query, conn)

In [38]:
fig = px.bar(
    df,
    x='genre',
    y='total_minutes',
    title='Общая длительность треков по жанрам',
    labels={'genre': 'Жанр', 'total_minutes': 'Длительность (минуты)'},
    color='genre',
    text='total_minutes',
    height=600)

In [40]:
fig

2.На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.

In [43]:
query = text("""
    SELECT 
        name AS track_name,
        ROUND(milliseconds / 60000.0, 2) AS minutes,
        ROUND(bytes / (1024*1024.0), 2) AS megabytes
    FROM track
    WHERE milliseconds > 0 AND bytes > 0
""")
with set_connection() as conn:
    df = pd.read_sql(query, conn)

In [46]:
df

,track_name,minutes,megabytes
0,For Those About To Rock (We Salute You),5.73,10.65
1,Balls to the Wall,5.71,5.26
2,Fast As a Shark,3.84,3.81
3,Restless and Wild,4.20,4.13
4,Princess of the Dawn,6.26,6.00
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,4.78,4.50
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",2.32,2.18
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",1.11,1.13
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",3.69,3.50


In [54]:
fig = px.scatter(
    df,
    x='minutes',
    y='megabytes',
    title='Соотношение длительности и размера треков',
    labels={
        'minutes': 'Длительность (минуты)',
        'megabytes': 'Размер (мегабайты)'
    }
)


In [55]:
fig

3.На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.

In [62]:
query = text("""
    SELECT 
        billing_country AS country,
        total
    FROM invoice
    WHERE total > 0
""")
with set_connection() as conn:
    df = pd.read_sql(query, conn)



In [63]:
fig = px.box(
    df,
    x='country',
    y='total',
    title='Распределение доходов по странам',
    labels={
        'country': 'Страна',
        'total': 'Сумма заказа ($)'
    },
    color='country',
    height=600,
    width=1000
)


In [64]:
fig

4.На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.

In [69]:
query = text("""
    SELECT 
        TO_CHAR(invoice_date, 'YYYY-MM') AS month,
        SUM(total) AS revenue
    FROM invoice
    WHERE EXTRACT(YEAR FROM invoice_date) = 2025
    GROUP BY TO_CHAR(invoice_date, 'YYYY-MM'), EXTRACT(MONTH FROM invoice_date)
    ORDER BY EXTRACT(MONTH FROM invoice_date)
""")

# Получение данных
with set_connection() as conn:
    df = pd.read_sql(query, conn)


In [71]:
df['month_name'] = pd.to_datetime(df['month']).dt.strftime('%B')

In [73]:
fig = px.bar(
    df,
    x='month_name',
    y='revenue',
    title='Динамика доходов по месяцам 2025 года',
    labels={
        'month_name': 'Месяц',
        'revenue': 'Доход ($)'
    },
    color='revenue',
    color_continuous_scale='Bluered',
    height=600
)

In [74]:
fig

5.На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.


In [77]:
query = text("""
    SELECT 
        CASE 
            WHEN billing_country = 'USA' THEN 'USA'
            ELSE 'Rest of World' 
        END AS region,
        ROUND(SUM(total), 2) AS total_sales
    FROM invoice
    GROUP BY 1
    ORDER BY 2 DESC
""")

# Получение данных
with set_connection() as conn:
    df = pd.read_sql(query, conn)

In [79]:
fig = px.pie(
    df,
    names='region',
    values='total_sales',
    title='Соотношение продаж: США vs Остальной мир',
    color='region',
    color_discrete_map={
        'USA': '#1f77b4',
        'Rest of World': '#ff7f0e'
    },
    hole=0.3,
    height=600
)


In [80]:
fig